In [8]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

## Prepare and Load Data

In [10]:
data = pd.read_csv('./data/boston_housing.csv')

X = data.drop('medv', axis=1).values
y = data['medv'].values

## PyTorch Linear Regression

In [11]:
class PyTorchLinearRegression:
    ''' Class that implemnets Multiple Linear Regression with PyTorch'''
    def __init__(self, num_of_features):
        self.w = torch.zeros(num_of_features, requires_grad=True)
        self.b = torch.zeros(1, requires_grad=True)
        self.learning_rate = 0.000001
    
    def _model(self, X):
        return X @ self.w.t() + self.b
    
    def _mse(self, pred, real):
        difference = pred - real
        return torch.sum(difference * difference) / difference.numel()
    
    def fit(self, X, y, epochs):
        X = torch.from_numpy(X).float()
        y = torch.from_numpy(y).float()
        
        for i in range(epochs):
            predictions = self._model(X)
            loss = self._mse(predictions, y)

            if i % 1000 == 0:
                print(f'Epoch: {i} - Loss: {loss}')

            loss.backward()
            with torch.no_grad():
                self.w -= self.w.grad * self.learning_rate
                self.b -= self.b.grad * self.learning_rate
                self.w.grad.zero_()
                self.b.grad.zero_()
    
    def predict(self, X):
        X = torch.from_numpy(X).float()
        return self._model(X)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

torch_model = PyTorchLinearRegression(X_train.shape[1])
torch_model.fit(X_train, y_train, 10000)

torch_predictions = torch_model.predict(X_test)
torch_predictions = torch_predictions.detach().numpy()
pd.DataFrame({
    'Actual Value': y_test,
    'Torch Prediction': torch_predictions,
})

Epoch: 0 - Loss: 608.6907958984375
Epoch: 1000 - Loss: 74.8147201538086
Epoch: 2000 - Loss: 71.41828918457031
Epoch: 3000 - Loss: 69.01657104492188
Epoch: 4000 - Loss: 67.03199768066406
Epoch: 5000 - Loss: 65.30267333984375
Epoch: 6000 - Loss: 63.767539978027344
Epoch: 7000 - Loss: 62.3945198059082
Epoch: 8000 - Loss: 61.1615104675293
Epoch: 9000 - Loss: 60.050819396972656


,Actual Value,Torch Prediction
0,20.5,20.263393
1,5.6,18.881317
2,13.4,1.071071
3,12.6,23.273228
4,21.2,21.131359
...,...,...
97,25.0,16.815962
98,19.5,22.625612
99,19.9,23.244698
100,15.4,18.252171
